<a href="https://colab.research.google.com/github/fedorov/IDC-Examples/blob/reorg-intro/notebooks/getting_started/part2_searching_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting started with IDC - Part 1: Setting up the prerequisites

## Summary

This notebook is part of [the series](link) introducing NCI Imaging Data Commons to the users who want to interact with IDC programmatically.

In this notebook you will learn how to set up your google account to be able to use Google Cloud Platform (GCP), which is the location of IDC data.

Initial version: Nov 2022

Updated: 

## Why Google Cloud Platform (GCP)?

GCP is a cloud-based environment that provides access to a suite of tools and services that include compute, storage and database resources, to name a few.

IDC is built upon the services provided by GCP, and stores its data within GCP. In order to fully leverage capabilities of IDC, you will need to activate GCP for your google account.

## GCP is not a free service - do I need to pay to use IDC?

No!

None of the activities in this tutorial seris will require you to pay for use of any GCP services, to have cloud credits, or even to connect your credit card to your account.

Egress of IDC data out of the cloud is free. While query of the data is not free, GCP [BigQuery free tier](https://cloud.google.com/bigquery/pricing#free-tier) includes 1 TB of query data per month, which will be sufficient to do a lot of queries of IDC data.

## What do I need to get started?

All you need is a Google account (identity) and a web browser. If you don't have a Google account, you can learn how to get one [here](https://accounts.google.com/signup/v2/webcreateaccount?dsh=308321458437252901&continue=https%3A%2F%2Faccounts.google.com%2FManageAccount&flowName=GlifWebSignIn&flowEntry=SignUp#FirstName=&LastName=). Note that you do NOT need a Gmail email account - [you can use your non-Gmail email address to create one instead](https://support.google.com/accounts/answer/27441?hl=en#existingemail).

<font color='red'>**WARNING**</font>: if you have a Google account that was provided by your organization, it may not be suitable for this tutorial due to the restrictions imposed by your organization. 